## IT333 - Tópicos em Eletrônica de Potência II
- **Professor:** José Antenor
- **Grupo:** 5
- **Integrantes:**
  - Lucas Pinheiro 182340
  - Rafael Afonso Veiga da Silva 272294
  - Alvaro Arturo Zarate Suarez 272300
  - Felipe Santiago Pereira de Oliveira 104895
- **Projeto:** Construção de Indutor para Filtro LCL (300uH)
---


### Parametros do inversor

| Parâmetro | Valor |
|----------|----------|
| Potencia aparente (S)| 1kVA | 
| Tensão (Vc = Vg)| 127V|
| Frequência (fg) | 60Hz |
| Corrente máxima ($I_{max}$)  | 11.135A $_{peak}$ |
| Ripple máximo ($\Delta I_{max}$) | 1.113A |
| Topologia da ponte | Full Bridge |
| Indice de modulação | 0.8 |
| Tensão CC ($V_{DC}$) | 225V |
| Freq. chaveamento(fsw) | 24kHz |
| Freq. amostragem(fs) | 48kHz |


### Dimensionando número de espiras

Para a construção do indutor, começamos com um nucleo já pré determinado e fornecido. Iremos usar o indutor de _part number_ [MMT034T10233](https://www.magmattec.com.br/wp-content/uploads/2019/03/MATERIAL-034_01.pdf).

Para a construção do indutor de 300uH, iremos usar a formula a seguir, que nos fornece o numero de espiras com base na indutância desejada e no parametro _Al_ disponivel no _datasheet_ do nucleo.

$$N=\sqrt{\frac{L}{Al}} \tag{1}$$

De acordo com o datasheet, $Al=110nH/esp^2$, assim teremos;

In [57]:
import math
Al = 110e-9
L = 300e-6

In [58]:
N = math.sqrt(L/Al)
N

52.22329678670935

Vamos arredondar o **número de espiras para 52** e seguir com a construção do indutor. A próxima fórmula que iremos usar também serve para encontrar o número de espiras, porém, usaremos para encontrar o valor da densidade de campo $B_m$.

$$N=\frac{L \cdot I_{pico}}{B_m \cdot A_e} \cdot 10^4 \tag{2}$$

O fator $10^4$ usado na equação serve para que possamos usar a área de secção do núcleo em $cm^2$. Caso o valor esteja em $m^2$, não é necessário o uso do fator multiplicativo.

A corrente de pico que iremos usar, será a corrente máxima somada à metade do ripple de corrente,


In [59]:
I_max = 11.135
I_ripple = 1.113

I_pico = I_max + I_ripple/2
I_pico

11.6915

A corrente de pico será então equivalente a $11.7\,A$. 

A área efetiva do núcleo toroidal é dada pela folha de dados do fabricante e, para o nosso caso, essa tem o valor de 6.85 $cm^2$

Resolvendo para $B_m$ em (2), o valor de densidade de campo será,

In [60]:
import sympy as sp

A_e = 6.85
B_m = sp.symbols("B_m")

B_m = sp.solve(sp.Eq((L*I_pico*10**4)/(B_m*A_e),N), B_m)
B_m

[0.0980475243532838]

O valor de densidade de campo de operação será de $0.0981\,T$


Após consultar a folha de dados do núcleo em uso, identificamos que a densidade de campo máxima $B_{max}$​ é de 11 000 Gauss, o que corresponde a 1.1 Tesla. Portanto, estamos operando dentro de uma faixa de densidade de campo que o núcleo pode suportar, sem entrar na região de saturação. É recomendável que esta utilização fique entre 50% e 70% da capacidade máxima do núcleo, para evitar qualquer problema que operar em saturação possa causar. No nosso caso, estamos utilizando uma proporção de uso de:

In [61]:
B_m_max = 1.1

B_relation_to_max = float(B_m[0]/1.1)
B_relation_to_max


0.08913411304843985

A faixa de uso em relação a $B_{max}$ é de apenas $8.9\%$, ou seja, nosso nucleo está super dimensionado para a nossa aplicação. De qualquer forma, isso não irá causar problemas.

Prosseguindo com os cálculos, iremos agora calcular a área da janela $(A_j)$ usada pelo indutor. Vamos considerar o número de espiras $(N)$ encontrado, uma densidade de corrente máxima $(J_{máx})$ de 4.5 $A/mm^2$. O fator de ocupação $(K_w)$ para núcleos toroidais fica abaixo de 0.3. Usaremos $K_w = 0.2$. Assim, teremos para $A_j$:

$$
A_j = \frac{N \cdot I_{ef}}{J_{max} \cdot K_w} \tag{3}
$$

In [62]:
K_w = 0.2
J_max =  4.5/0.01 #converting from A/mm^2 to A/cm^2
I_ef = I_pico/math.sqrt(2)

Aj = (N*I_ef)/(J_max*K_w)
Aj

4.797080555949561

Temos assim, $A_j = 4.8 cm^2$.
No datasheet do núcleo, podemos verificar a área de janela disponível, uma vez que temos acesso ao diametro interno do toróide, 5.72 $cm$, temos assim:

In [63]:
math.pi * (5.72/2)**2

25.696971269303067

A área de janela disponível $(A_{j\:disponivel})$ do nucleo toroidal que possuímos é de $25.7\:cm^2$. 

Podemos concluir então que, a área disponível, é mais que suficiente para a realização do enrolamento do fio no núcleo fornecido. $A_{j\:usado} < A_{j\:disponivel}$

### Dimensionamento do condutor

O condutor a ser usado na construção do indutor precisará ter secção capaz de suportar a corrente de operação do circuito. Para isso usaremos a fórmula (4) para encontrar qual a secção ideal deste fio,

$$S_f = \frac{I_{ef}}{J_{máx}} \tag{4}$$

In [64]:
S_f = I_ef/(J_max*0.01) #multiplying by 0.01 to convert from cm^2 to mm^2
S_f

1.837141984942782

A secção ideal deste fio é então $S_f = 1.83\:mm^2$.

O próximo passo é aferir qual o impacto da frequência de chaveamento na seleção deste condutor. Para isso iremos usar a fórmula (5) para saber qual profundidade a corrente irá fluir no fio. Caso a secção deste condutor seja menor do que $S_f$, será necessário utilizar mais de um condutor em paralelo até atingir uma secção igual ou superior a de $S_f$.

$$\gamma = \sqrt{\frac{4.35 \cdot 10^{-3}}{f}} \tag{5}$$

In [65]:
freq = 48e3
gamma = math.sqrt(4.35e-3/freq)
gamma

0.0003010398644698074

Obtemos então que, $\gamma = 0.3\:mm$. Isso implica que, podemos usar um fio com diâmetro de $0.6\,mm$ sem desperdício algum de material.

Tivemos acesso à um fio esmaltado de 18 AWG. De acordo com as tabelas consultadas online, o fio possui diametro de $1.024\:mm$. Assim, haverá uma pequena região circular, no interior deste fio, que não será de fato útil para a passagem de corrente em $48kHz$, porém, como material e dimensões não são restrições para o projeto, podemos seguir sem maiores problemas.

Usando o diâmetro do cabo de 18AWG encontrado para obter a secção, temos:

In [66]:
math.pi*(1.024/2)**2

0.8235496645826427

secção do cabo de 18 AWG é de $0.823\:mm^2$.

<div style="text-align: center;">
  <img src="img/FFF_Corrente_JPG (1).jpg" alt="Indutância" width=70%><br>
  <figcaption>Fig.1 - Simulação da FFT de corrente do modelo disponibilizado no PSIM.</figcaption>
</div>
<br>

Podemos observar que, de acordo com a simulação, a raia majoritária no espectro de corrente é a que corresponde a frequencia de 60Hz. Isso faz com que o impacto do efeito pelicular no cabo usado seja bem baixo, corroborando assim com uma boa escolha de secção para a aplicação.

Portanto, para este fio, conseguimos conhecer o número de condutores em paralelo $(ncp)$ necessário para a construção do indutor. Calculamos $ncp$ a seguir:

$$ncp = \frac{S_f}{S_{18\:awg}} \tag{6} $$

In [67]:
S_f_18 = (math.pi*(1.024/2)**2)
ncp = S_f/S_f_18
ncp

2.2307604069923412

Com $ncp = 2.23$, podemos concluir que, para obter a secção de condutor ideal precisaremos usar 3 fios condutores e 52 voltas em torno do núcleo toroidal fornecido para a construção do indutor de $300uH$.

Utilizando o software aberto Finite Element Method Magnetics (FEMM) fizemos a simulação com os parâmetros **disponibilizados** e chegamos na indutância de $340uH$. 

<div style="text-align: center;">
  <img src="img/inductance.png" alt="Indutância" width=60%><br>
  <figcaption>Fig.2 - Simulação de indutância.</figcaption>
</div>
<br>
<br>
As imagens a seguir também nos mostram a distribuição teórica das densidades de campo (Fig.3) e da densidade de fluxo (Fig.4) ao longo do corpo do núcleo toroidal. É interessante observar que o fluxo tende a ser maior quanto mais próximo do raio interno do toróide. Isso é justificado pelo fato de esta região apresentar um caminho menor, implicando assim em uma menor relutância da região. 
<br>
<br>
<div style="text-align: center;">
  <img src="img/FieldDensity.png" alt="Indutância" width=60%><br>
  <figcaption>Fig.3 - Simulação de densidade de campo.</figcaption>
</div>
<br>
<br>
<br>
<div style="text-align: center;">
  <img src="img/FluxDensity.png" alt="Indutância" width=60%><br>
  <figcaption>Fig.4 - Simulação de densidade de fluxo.</figcaption>
</div>
<br>
<br>

A intensidade de campo de operação $H_m$ pode ser encontrada a partir de $B_m$ e da permeabilidade magnética do nucleo encontrada no datasheet.

In [68]:
u0 = 4*math.pi*1e-7 #vacuum magnetic permeability
uc = 33*u0
H_m = float(B_m[0])/uc #intensidade de campo no ponto de operação 
print(f"Intensidade de campo no ponto de operação: {H_m:.4f} A/m")

Intensidade de campo no ponto de operação: 2364.3558 A/m


As simulações nos mostram distribuições e gradientes ao longo do núcleo do indutor e, os valores encontrados de $B_m = 0.098\,T$ e $H_m=2.36e3\,A/m$ são valores médios que estão contidos dentro das distribuições encontradas.

### Cálculo de perdas

O fio usado na construção irá apresentar algumas perdas devido à sua resistência não ideal. Essa perda devido a resistência pode ser estimada encontrando a resistência total do fio 


In [69]:
# Wire loss
rho_cu = 17.241;    # Copper resistivity at 20°C [Ohm*mm2/km]
L = 7e-3; #Length [km]
ncp = 3
R = rho_cu*L/(S_f_18*ncp)
perda_wire = (I_ef**2)*R
print(f"Perdas no fio: {perda_wire:.6f} W")

Perdas no fio: 3.338566 W


<div style="text-align: center;">
  <img src="img/core_loss_plot.jpeg" alt="plot_perdas" width=60%><br>
  <figcaption>Fig.5 - Gráfico de perdas no núcleo vs densidade de fluxo que foi diusponibilizado no material.</figcaption>
</div>
<br>
<br>

Com o auxílio do gráfico acima, identificamos com o auxílio da Fig.1 que a raia de $60Hz$ é a principal no espectro de corrente, assim, ela será responsável pela maior parcela das perdas.

Realizamos uma interpolação dos limites apresentados na Fig.5 e pegamos o valor de perda para a região de operação do nosso circuito.

In [70]:
import numpy as np
import scipy.interpolate as interp

r_int = 5.72 #in cm
r_ext = 10.2 #in cm
thickness_toroide = 3.3 # height in cm
Aj_toroide = math.pi*(r_int/2)**2
Area_ext_radius = math.pi*(r_ext/2)**2
V_toroide = (Area_ext_radius-Aj_toroide)*thickness_toroide #volume in cm^3

# Given data
Perdas = np.array([10.08131068, 11.56927059, 61.84786841, 163.4423491, 322.6951938, 546.2575102, 839.0703189, 1217.812156, 1643.271266, 2164.149315, 2759.28898])  # [mW/cm^3]
Points_Bm = 1e-4 * np.array([944.8813643, 998.1727714, 2003.673003, 3007.165067, 4000.047339, 5009.123227, 6003.376101, 7000.276121, 8029.458521, 9010.024522, 10000])  # [T]

# Create a lookup table
tabela_busca = np.column_stack((Points_Bm, Perdas))
tabela_busca = tabela_busca[tabela_busca[:, 0].argsort()]

Bm = float(B_m[0])

# Interpolate 
perdas_interp = interp.interp1d(tabela_busca[:, 0], tabela_busca[:, 1], kind='linear')
perda_core = perdas_interp(Bm) * (V_toroide / 1e3)  # Perdas totais no core [W]

print(f"Perdas_core: {perda_core:.6f} W")


Perdas_core: 2.047264 W


Assim, as perdas totais do indutor serão as perdas associadas ao núcleo somadas às perdas associadas ao condutor,

$$
Perda_{tot} = Perda_{wire}+Perda_{core} \tag{7}
$$

In [71]:
perda_total = perda_wire + perda_core
print(f"Perdas totais: {perda_total:.6f} W")

Perdas totais: 5.385830 W


Então, as perdas totais relacionadas ao núcleo e ao fio equivalem a um total de cerca de $5.4W$

Não conseguimos por nenhuma fonte o valor da resistência térmica do núcleo de pó de ferro, logo, não foi possível estimar a temperatura de operação do indutor para as condições calculadas.